In [1]:
import torch 
import numpy as np
from botorch.models import SingleTaskGP
from botorch  import fit_gpytorch_model

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf

from sklearn.gaussian_process import GaussianProcessRegressor
from scipy.stats import norm
from scipy.optimize import minimize
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF


In [2]:
f8_inputs = np.load("initial_inputs.npy")
f8_outputs = np.load("initial_outputs.npy")
f8_inputs_2 = np.load("initial_inputs2.npy")
f8_outputs_2 = np.load("initial_outputs2.npy")

In [3]:
f8_inputs = np.concatenate((f8_inputs, f8_inputs_2)) 

f8_inputs = np.concatenate((f8_inputs, np.array([[0.084259,0.384768,0.152956,0.83177,0.619797,0.838113,0.360231,0.819888]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.265797,0.781385,0.799061,0.737909,0.541671,0.059651,0.661726,0.249156]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.085447,0.080955,0.045928,0.041786,0.373935,0.831055,0.468307,0.923084]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.010576,0.090955,0.105928,0.061786,0.353935,0.891055,0.478307,0.943084]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.065447,0.080955,0.055928,0.051786,0.383935,0.841055,0.478307,0.913084]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.035447,0.080955,0.045928,0.041786,0.373935,0.831055,0.468307,0.923084]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.040447,0.080955,0.045928,0.041786,0.373935,0.831055,0.468307,0.923084]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.015447,0.080955,0.080928,0.041786,0.373935,0.866055,0.468307,0.923084]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.154598,0.138109,0.018722,0.059536,0.367817,0.883297,0.437297,0.865776]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.525834,0.680905,0.197024,0.703188,0.541326,0.930864,0.505922,0.13963]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.151301,0.185001,0.182801,0.197001,0.631301,0.538201,0.257201,0.594101]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.150301,0.194801,0.185401,0.194301,0.601801,0.534401,0.269901,0.604701]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.144501,0.190901,0.128601,0.104101,0.187601,0.654601,0.501901,0.841001]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.147401,0.209901,0.130901,0.128401,0.191901,0.630201,0.487101,0.802801]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.151301,0.085001,0.182801,0.197001,0.731301,0.538201,0.257201,0.394101]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.251301,0.185001,0.182801,0.297001,0.831301,0.538201,0.257201,0.494101]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.351301,0.285001,0.382801,0.397001,0.631301,0.538201,0.457201,0.494101]])))
f8_inputs = np.concatenate((f8_inputs, np.array([[0.251301,0.185001,0.182801,0.197001,0.731301,0.538201,0.357201,0.494101]])))

f8_outputs = np.concatenate((f8_outputs,f8_outputs_2)) 
f8_outputs = np.append(f8_outputs,9.2912571799561)
f8_outputs = np.append(f8_outputs,7.1918457066059)
f8_outputs = np.append(f8_outputs,9.6071159102679)
f8_outputs = np.append(f8_outputs,9.5519090503339)
f8_outputs = np.append(f8_outputs,9.5991537082679)
f8_outputs = np.append(f8_outputs,9.5992053102679)
f8_outputs = np.append(f8_outputs,9.6004463702679)
f8_outputs = np.append(f8_outputs,9.5828223402679)
f8_outputs = np.append(f8_outputs,9.5885735221739)
f8_outputs = np.append(f8_outputs,8.608506577299)
f8_outputs = np.append(f8_outputs,9.9607020326694)
f8_outputs = np.append(f8_outputs,9.9511627380494)
f8_outputs = np.append(f8_outputs,9.5927395277894)
f8_outputs = np.append(f8_outputs,9.6206379181294)
f8_outputs = np.append(f8_outputs,9.9653361726694)
f8_outputs = np.append(f8_outputs,9.9134032526694)
f8_outputs = np.append(f8_outputs,9.4536206526694)
f8_outputs = np.append(f8_outputs,9.8880531526694)


In [4]:
noise_level = 9
np.random.seed(42)
def function8(x1,x2,x3,x4,x5,x6,x7,x8, noise_level=noise_level):
    return np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2)) + \
        np.sin(1 * x3+4*x4) * (1 - np.tanh(x3 ** 2) - np.tanh(x4)) + \
        np.sin(1 * x5+4*x6) * (1 - np.tanh(x5 ** 2) - np.tanh(x6)) + \
        np.sin(1 * x7+4*x8) * (1 - np.tanh(x7 ** 2) - np.tanh(x8)) + \
               + np.random.randn() * noise_level

In [5]:
variable_value = np.linspace(1, 10, 10)
n_dims = 8
beta = 1.1818181818181819
np.random.seed(42)
train_X = torch.tensor(f8_inputs)
train_Y = torch.tensor(f8_outputs).reshape(len(f8_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level =6.909090909090909


for i in range(len(variable_value)):
   # noise_level = variable_value[i]
   
    def function_8(x1,x2,x3,x4,x5,x6,x7,x8, noise_level=noise_level):
        return np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2)) + \
            np.sin(1 * x3+4*x4) * (1 - np.tanh(x3 ** 2) - np.tanh(x4)) + \
            np.sin(1 * x5+4*x6) * (1 - np.tanh(x5 ** 2) - np.tanh(x6)) + \
            np.sin(1 * x7+4*x8) * (1 - np.tanh(x7 ** 2) - np.tanh(x8)) + \
                   + np.random.randn() * noise_level
    
    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    UCB = UpperConfidenceBound(gp, beta=beta)

    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=50, raw_samples=200,
    )


    x1,x2,x3,x4,x5,x6,x7,x8 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = function_8(x1,x2,x3,x4,x5,x6,x7,x8)

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2,x3,x4,x5,x6,x7,x8


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    print(f"beta :{beta} noise_level :{noise_level}")
    print(f"Next point: {candidate} \n Next Y {y_next}")
    print(f"best_x:{best_x}Best_y,{best_y}")
    

beta :1.1818181818181819 noise_level :6.909090909090909
Next point: tensor([[0.0898, 0.1392, 0.1343, 0.1018, 0.8807, 0.5058, 0.1850, 0.6032]]) 
 Next Y tensor([4.7996], dtype=torch.float64)
best_x:tensor([0.1513, 0.0850, 0.1828, 0.1970, 0.7313, 0.5382, 0.2572, 0.3941],
       dtype=torch.float64)Best_y,9.9653361726694
beta :1.1818181818181819 noise_level :6.909090909090909
Next point: tensor([[0.1967, 0.0090, 0.1695, 0.3188, 0.6655, 0.6384, 0.2562, 0.2336]]) 
 Next Y tensor([1.1667], dtype=torch.float64)
best_x:tensor([0.1513, 0.0850, 0.1828, 0.1970, 0.7313, 0.5382, 0.2572, 0.3941],
       dtype=torch.float64)Best_y,9.9653361726694
beta :1.1818181818181819 noise_level :6.909090909090909
Next point: tensor([[0.1503, 0.2974, 0.2232, 0.3937, 0.7542, 0.4959, 0.2058, 0.4356]]) 
 Next Y tensor([6.1971], dtype=torch.float64)
best_x:tensor([0.1513, 0.0850, 0.1828, 0.1970, 0.7313, 0.5382, 0.2572, 0.3941],
       dtype=torch.float64)Best_y,9.9653361726694
beta :1.1818181818181819 noise_level :6.

In [ ]:

beta :1.1818181818181819 noise_level :6.909090909090909
Next point: tensor([[0.1546, 0.1381, 0.0187, 0.0595, 0.3678, 0.8833, 0.4373, 0.8658]]) 
 Next Y tensor([10.2376], dtype=torch.float64)
best_x:(0.15459848940372467, 0.1381096988916397, 0.018722787499427795, 0.05953603237867355, 0.3678175210952759, 
        0.8832974433898926, 0.4372975826263428, 0.865776002407074)
    Best_y,10.237612055415791



beta :1.1818181818181819 noise_level :6.909090909090909
Next point: tensor([[0.0359, 0.0783, 0.1931, 0.2068, 0.2691, 0.7741, 0.4785, 0.8636]]) 
 Next Y tensor([5.4406], dtype=torch.float64)
best_x:tensor([0.0404, 0.0810, 0.0459, 0.0418, 0.3739, 0.8311, 0.4683, 0.9231],
       dtype=torch.float64)Best_y,9.6004463702679






beta :0.1 noise_level :6.909090909090909
Next point: tensor([[0.2516, 0.8380, 0.7831, 0.7467, 0.6140, 0.0713, 0.6979, 0.2553]]) 
 Next Y tensor([9.9653], dtype=torch.float64)
best_x:(0.2516104280948639, 0.8379758596420288, 0.7830837368965149, 0.7467195391654968, 0.613950252532959, 
        0.07130374014377594, 0.697936475276947, 0.255339115858078)Best_y,9.96533089251728

    
beta :1.2727272727272727 noise_level :6.909090909090909
Next point: tensor([[0.0377, 0.0918, 0.0075, 0.0113, 0.4444, 0.9188, 0.5478, 0.9660]]) 
 Next Y tensor([11.0236], dtype=torch.float64)
best_x:(0.03765999525785446, 0.09182529151439667, 0.0074640982784330845, 0.011280793696641922, 
        0.4443797171115875, 0.9187936186790466, 
        0.5477606058120728, 0.9660108089447021)Best_y,11.023583532446972

In [29]:
X_max= f8_inputs[np.argmax(abs(f8_outputs))]
print(X_max)

[0.085447 0.080955 0.045928 0.041786 0.373935 0.831055 0.468307 0.923084]


In [30]:
Y_max= f8_outputs[np.argmax(abs(f8_outputs))]
print(Y_max)

9.6071159102679


In [17]:
f8_inputs = np.delete(f8_inputs, np.argmax(abs(f8_outputs)), 0)
len(f8_inputs)

89

In [18]:
f8_outputs= np.delete(f8_outputs, np.argmax(abs(f8_outputs)), 0)
len(f8_outputs)

89

In [6]:
def costly_function(x):
    total = np.array([])
    for x_i in x:
         #total = np.append(total, np.sum(np.exp(-(x_i - 5) ** 2)))
        total = np.exp(-(x_i - 2) ** 2)+np.exp(-(x_i + 6) ** 2/10)+np.exp(-(x_i - 1) ** 3/9)
    return total + np.random.randn()

In [7]:
variable_value = np.linspace(1, 10, 10)
n_dims = 8
beta =1.1818181818181819
np.random.seed(42)
train_X = torch.tensor(f8_inputs)
train_Y = torch.tensor(f8_outputs).reshape(len(f8_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level =6.909090909090909
candidate_list = []
best_x_list= []

for i in range(len(variable_value)):
   # noise_level = variable_value[i]
   
    
    
    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    UCB = UpperConfidenceBound(gp, beta=beta)

    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=500, raw_samples=1000,
    )


    x1,x2,x3,x4,x5,x6,x7,x8 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = costly_function([x1,x2,x3,x4,x5,x6,x7,x8])

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2,x3,x4,x5,x6,x7,x8


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    candidate_list.append(candidate) 
    best_x_list.append(best_x)
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    #print(f"beta :{beta} noise_level :{noise_level}")
    #print(f"Next point: {candidate} \n Next Y {y_next}")
    #print(f"best_x:{best_x}Best_y,{best_y}")
    

In [8]:
candidate_list

[tensor([[0.0898, 0.1392, 0.1343, 0.1018, 0.8807, 0.5058, 0.1850, 0.6031]]),
 tensor([[0.1878, 0.0687, 0.2057, 0.2774, 0.6714, 0.6135, 0.3121, 0.2382]]),
 tensor([[0.2323, 0.3124, 0.2046, 0.2894, 0.4161, 0.5569, 0.4375, 0.7228]]),
 tensor([[0.2448, 0.2133, 0.2172, 0.3442, 0.7818, 0.5187, 0.2748, 0.4415]]),
 tensor([[0.0572, 0.0751, 0.0762, 0.0236, 0.3017, 0.7912, 0.4670, 0.9060]]),
 tensor([[0.0647, 0.1079, 0.0396, 0.0781, 0.4284, 0.8965, 0.4821, 0.9183]]),
 tensor([[0.0929, 0.1266, 0.1864, 0.0362, 0.1793, 0.4614, 0.4070, 0.8181]]),
 tensor([[0.2772, 0.7730, 0.3310, 0.4666, 0.6293, 0.6778, 0.3769, 0.2885]]),
 tensor([[0.1330, 0.0848, 0.2997, 0.6088, 0.1268, 0.1458, 0.6523, 0.1820]]),
 tensor([[0.1421, 0.1029, 0.2838, 0.2337, 0.2159, 0.9496, 0.5344, 0.8146]])]

In [28]:
best_x_list

[tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572, 0.5941],
        dtype=torch.float64),
 tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572, 0.5941],
        dtype=torch.float64),
 tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572, 0.5941],
        dtype=torch.float64),
 tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572, 0.5941],
        dtype=torch.float64),
 tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572, 0.5941],
        dtype=torch.float64),
 tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572, 0.5941],
        dtype=torch.float64),
 tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572, 0.5941],
        dtype=torch.float64),
 tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572, 0.5941],
        dtype=torch.float64),
 tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572, 0.5941],
        dtype=torch.float64),
 tensor([0.1513, 0.1850, 0.1828, 0.1970, 0.6313, 0.5382, 0.2572,

In [ ]:
[0.04343225 0.10004702 0.22342261 0.01686426 0.28000728 0.36384718,0.32840638 0.83239639]